In [1]:
import sys
sys.path.append("../repo/KLens/")
sys.path.append("../repo/kl_measurement/")

In [2]:
import numpy as np
import os
import sys
import copy
from astropy.io import fits
import galsim
from scipy.interpolate import interp1d
from scipy.ndimage.interpolation import rotate
import tfCube
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.backends.backend_pdf import PdfPages

In [10]:
def getSlitSpectra(data=None, pars=None):
    #weights = getSlitWeights(pars)
    spectra = []
    extent = pars['image_size'] * pars['pixScale']
    subGridPixScale = extent*1./pars['ngrid']
    grid = np.arange(-extent/2., extent/2., subGridPixScale)
    xx,yy = np.meshgrid(grid,grid)
    slit_weight = np.ones((pars['ngrid'],pars['ngrid']))
    slit_weight[np.abs(yy-pars['slitOffset']) > pars['slitWidth']/2.] = 0.
        
    for this_slit_angle in pars['slitAngles']:
        this_data = rotate(data, -this_slit_angle*(180./np.pi),reshape=False)
        spectra.append(np.sum(this_data*slit_weight[:,:,np.newaxis],axis=0))
    return spectra

In [4]:
pars = tfCube.getParams(redshift = 0.2)
pars['type_of_observation'] = 'slit'
# to make things practical during testing, increase the spaxel size.
pars['g1'] = 0.0
pars['g2'] = 0.0
pars['nm_per_pixel'] = 0.025
pars['expTime'] = 10000.
pars['pixScale'] = 0.032
pars['Resolution'] = 5000
pars['sini'] = 0.5
pars['aspect'] = 0.2
pars['vcirc'] = 200.
pars['area'] = 3.14*(1000./2.)**2
pars['linelist']['flux'][pars['linelist']['species'] == 'Halpha'] = 6e-24
pars['norm'] = 6e-24
pars['lambda_min'] = (1 + pars['redshift']) * pars['linelist']['lambda'][pars['linelist']['species'] == 'Halpha'] - 2
pars['lambda_max'] = (1 + pars['redshift']) * pars['linelist']['lambda'][pars['linelist']['species'] == 'Halpha'] + 2

pars['knot_fraction']=0.

lines = pars['linelist']
pars['half_light_radius'] = 0.5
#lines['flux'] = 1e-25 * 1e-9 # We seem to need another factor of 1e-9 here.
#pars['linelist'] = lines
pars['slitAngles'] = np.array([0.])#np.linspace(-np.pi/4., np.pi/2., 3)
pars['slitWidth']  = 0.3
pars['slitOffset'] = 0.0
# define some fiber parameters
#nfiber = 5
#r_off = 1.5
pars['fiber_size'] = 1.0
pars['psfFWHM'] = .25
pars['vscale'] = pars['half_light_radius']
pars['ngrid'] = 256
pars['image_size'] = 128

extent =  pars['image_size'] * pars['pixScale']
subGridPixScale = extent*1./pars['ngrid']

In [5]:
aperture = galsim.Image(pars['ngrid'], pars['ngrid'],scale=subGridPixScale)

In [6]:
obsLambda, obsGrid, modelGrid, skyGrid = tfCube.getTFcube(pars,aperture,[0.,0.])

returning:
lambda, observation, model, sky (the last three are (npix, npix, nspax) datacubes)


In [23]:
lambda_cen = (pars['lambda_min']+pars['lambda_max'])/2.0
lambda0=656.461
c = 299792.458
v_cen1 =(lambda_cen-lambda0)/lambda0*c
v_cen1

array([59958.4916])

In [25]:
v_cen2=0.2*c
v_cen2

59958.4916

In [7]:
def velocity_mimMAX(lambda0,redshift,lambda_minMAX):
    c = 299792.458
    lambda_cen = lambda0*(1+redshift)
    v_cen      = redshift*c
    v_min      = (lambda_minMAX[0]-lambda0)/lambda0*c - v_cen
    v_MAX      = (lambda_minMAX[1]-lambda0)/lambda0*c - v_cen
    return v_min,v_MAX

In [8]:
v_min,v_MAX = velocity_mimMAX(lambda0=656.461,redshift=0.2,lambda_minMAX=[np.min(obsLambda),np.max(obsLambda)])

In [9]:
v_min,v_MAX

(-913.3595384950677, 901.9425442622014)


# slitOffset

In [11]:
pars['slitOffset'] = 0.0
pars['slitWidth']  = 0.3
data_00 = getSlitSpectra(data=modelGrid,pars=pars)

In [12]:
pars['slitOffset'] = 0.5
pars['slitWidth']  = 0.3
data_0i = getSlitSpectra(data=modelGrid,pars=pars)

In [20]:
%matplotlib

ncol=3

fig,ax = plt.subplots(1,ncol,figsize=(20,4))
#ax[0].imshow(np.sum(modelGrid,axis=2))
ax[0].imshow(data_00[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale'] ], aspect='auto')
ax[1].imshow(data_0i[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale']], aspect='auto')
ax[2].imshow((data_00[0]-data_0i[0])/data_00[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale']], aspect='auto')

#ycen = (pars['lambda_min']+pars['lambda_max'])/2.
vcen = 0.


for j in range(ncol):
    ax[j].axhline(y=0.0,color='white', linestyle='-',lw=1)
    ax[j].axhline(y=0.5,color='white', linestyle=':',lw=1)
    ax[j].axhline(y=-0.5,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen,color='white', linestyle='-',lw=1)
    ax[j].axvline(x=vcen+100,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen-100,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen+200,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen-200,color='white', linestyle=':',lw=1)
    
plt.show()

Using matplotlib backend: MacOSX


# slitWidth

In [14]:
pars['slitOffset'] = 0.0
pars['slitWidth']  = 0.03
data_00 = getSlitSpectra(data=modelGrid,pars=pars)

In [15]:
pars['slitOffset'] = 0.0
pars['slitWidth']  = 0.6
data_0i = getSlitSpectra(data=modelGrid,pars=pars)

In [16]:
%matplotlib

ncol=3

fig,ax = plt.subplots(1,ncol,figsize=(20,4))
#ax[0].imshow(np.sum(modelGrid,axis=2))
ax[0].imshow(data_00[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale']], aspect='auto')
ax[1].imshow(data_0i[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale']], aspect='auto')
ax[2].imshow((data_00[0]-data_0i[0])/data_00[0],extent=[v_min, v_MAX,-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale']], aspect='auto')

#ycen = (pars['lambda_min']+pars['lambda_max'])/2.
vcen = 0.

for j in range(ncol):
    ax[j].axhline(y=0.0,color='white', linestyle='-',lw=1)
    ax[j].axhline(y=0.5,color='white', linestyle=':',lw=1)
    ax[j].axhline(y=-0.5,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen,color='white', linestyle='-',lw=1)
    ax[j].axvline(x=vcen+100,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen-100,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen+200,color='white', linestyle=':',lw=1)
    ax[j].axvline(x=vcen-200,color='white', linestyle=':',lw=1)
    
plt.show()

Using matplotlib backend: MacOSX


In [132]:
data_00[0].shape

(256, 160)

In [133]:
data_00[0]

array([[19.24160819, 19.24282675, 19.24404536, ..., 19.43339562,
        19.43462024, 19.43584491],
       [19.99519809, 19.99646438, 19.99773071, ..., 20.1944968 ,
        20.19576939, 20.19704201],
       [20.77820961, 20.77952549, 20.78084141, ..., 20.98531285,
        20.98663527, 20.98795773],
       ...,
       [20.75502846, 20.75634287, 20.75765732, ..., 20.96190009,
        20.96322104, 20.96454202],
       [19.97286211, 19.97412698, 19.97539189, ..., 20.17193765,
        20.17320881, 20.17448001],
       [19.22008751, 19.22130471, 19.22252195, ..., 19.41165992,
        19.41288317, 19.41410646]])

In [19]:
%matplotlib
plt.imshow(data_00[0])
plt.show()

Using matplotlib backend: MacOSX


In [24]:
%matplotlib
fig,axes = plt.subplots(nrows=1,ncols=len(data)+1,figsize=(6*(len(data)+1),6))
axes[0].imshow(np.sum(modelGrid,axis=2))
for ispec,ax in zip(data,axes[1:]):
    ax.imshow(ispec,extent=[-pars['ngrid']/2*pars['pixScale'],pars['ngrid']/2.*pars['pixScale'], np.min(obsLambda), np.max(obsLambda)])
    ax.axvline(x=0.0,color='white', linestyle='-')

#fig.savefig('model_keck_spectra.pdf',format='pdf')
plt.show()

Using matplotlib backend: MacOSX
